
#  Divide-and-conquer algorithm


Enter your name and student ID.

 * Name:
 * Student ID:


# 1. Compilers

## 1-1. Set up NVIDIA HPC SDK
Execute this before you use NVIDIA HPC SDK

In [ ]:
export PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/22.11/compilers/bin:$PATH
#export PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/22.9/compilers/bin:$PATH
#export PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/22.7/compilers/bin:$PATH

Check if it works (check if full paths of nvc/nvc++ are shown)

In [ ]:
which nvc
which nvc++

## 1-2. Set up LLVM
Execute this before you use LLVM

In [ ]:
export PATH=/home/share/llvm/bin:$PATH

Check if it works (check if full paths of nvc/nvc++ are shown)

In [ ]:
which clang
which clang++


# 2. Merge sort
* merge sort is a divide-and-conquer algorithm that sorts an array as follows
1. split the array into two halves
1. recursively apply merge sort to each half
1. merge the two (sorted) arrays

* the merge step needs another array, so it takes two arrays (one is used for the input/output and the other used for workspace)

* here is a sequential merge sort algorithm

In [ ]:
%%writefile msort.cc
#com 2
#include "msort.h"

/* insertion sort for leaf cases */
/* return m a[m] = min {a[begin:end]} */ 
static long choose_min_idx(float * a, long begin, long end) {
  long m = begin;
  for (long i = begin + 1; i < end; i++) {
    if (a[i] < a[m]) {
      m = i;
    }
  }
  return m;
}

/* sort a[p:q] by insertion sort */
static void insertion_sort(float * a, long p, long q) {
  for (long i = p; i < q; i++) {
    long j = choose_min_idx(a, i, q);
    float t = a[i];
    a[i] = a[j];
    a[j] = t;
  }
}

/* sort a[p:q] into g[p:q], using b[p:q] as a temporary space */
static void msort_r(float * a, float * b, float * g, long p, long q, long th0, long th1) {
  if (q - p < th0) {
    /* the array is small -> switch to insertion sort */
    if (g != a) {
      for (long i = p; i < q; i++) {
        g[i] = a[i];
      }
    }
    insertion_sort(g, p, q);
  } else {
    long r = (p + q) / 2;
    /* get partial results into the other array != g */
    float * h = (g == a ? b : a);
#ifpy VER == 2
#pragma omp task
#endifpy
    msort_r(a, b, h, p, r, th0, th1);
#ifpy VER == 2
#pragma omp task
#endifpy
    msort_r(a, b, h, r, q, th0, th1);
#ifpy VER == 2
#pragma omp taskwait
#endifpy
    /* merge h[p:r] and h[r:q] -> g[p:]*/
    merge(h, g, p, r, r, q, p, th1);
  }
}

/* merge sort, called from the main */
void msort_from_main(float * a, float * b, float * g, long p, long q, long th0, long th1) {
#ifpy VER == 2
#pragma omp parallel
#pragma omp master
#endifpy
  msort_r(a, b, g, p, q, th0, th1);
}


* and a merge procedure

In [ ]:
%%writefile merge.cc
#com 2
#include <assert.h>
#include "msort.h"

#ifpy VER == 1
void merge(float * a, float * b, long p, long q, long s, long t, long d, long th) {
  (void)th;
  long i = p;
  long j = s;
  long k = d;
  while (i < q && j < t) {
    if (a[i] < a[j]) {
      b[k++] = a[i++];
    } else {
      b[k++] = a[j++];
    }
  }
  while (i < q) {
    b[k++] = a[i++];
  }
  while (j < t) {
    b[k++] = a[j++];
  }
}

/* merge, called from main */
void merge_from_main(float * a, float * b, long p, long q, long s, long t, long d, long th) {
  merge(a, b, p, q, s, t, d, th);
}

#elsepy

/* merge a[p:q] and a[r:s] --> b[d:] */
static void merge_s(float * a, float * b, long p, long q, long s, long t, long d) {
  long i = p;
  long j = s;
  long k = d;
  while (i < q && j < t) {
    if (a[i] < a[j]) {
      b[k++] = a[i++];
    } else {
      b[k++] = a[j++];
    }
  }
  while (i < q) {
    b[k++] = a[i++];
  }
  while (j < t) {
    b[k++] = a[j++];
  }
}

/* find r s.t. a[r-1] <= piv < a[r] */
static long find_piv(float * a, long b, long e, float piv) {
  if (piv < a[b]) return b;
  if (a[e - 1] <= piv) return e;
  long p = b;
  long q = e - 1;
  /* a[p] <= piv < a[q] */
  while (q - p > 1) {
    assert(a[p] <= piv);
    assert(piv < a[q]);
    long r = (p + q) / 2;
    if (a[r] <= piv) {
      p = r;                    // a[p] <= piv
    } else {
      q = r;                    // piv < a[q]
    }
  }
  assert(a[q - 1] <= piv);
  assert(piv < a[q]);
  /* a[q - 1] <= piv < a[q] */
  return q;
}

/* merge a[p:q] and a[r:s] --> b[d:] */
void merge(float * a, float * b, long p, long q, long s, long t, long d, long th) {
  if (q - p + t - s < th) {
    merge_s(a, b, p, q, s, t, d);
  } else if (q - p > t - s) {
    long r = (p + q) / 2;
    float piv = a[r];
    long u = find_piv(a, s, t, piv);
#pragma omp task
    merge(a, b, p, r, s, u, d, th);
#pragma omp task
    merge(a, b, r, q, u, t, d + (r - p) + (u - s), th);
#pragma omp taskwait
  } else {
    long u = (s + t) / 2;
    float piv = a[u];
    long r = find_piv(a, p, q, piv);
#pragma omp task
    merge(a, b, p, r, s, u, d, th);
#pragma omp task
    merge(a, b, r, q, u, t, d + (r - p) + (u - s), th);
#pragma omp taskwait
  }
}

/* merge, called from main */
void merge_from_main(float * a, float * b, long p, long q, long s, long t, long d, long th) {
#pragma omp parallel
#pragma omp master
  merge(a, b, p, q, s, t, d, th);
}

#endifpy

* here is the main program and the header file

In [ ]:
%%writefile msort_main.cc
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include "event.h"
#include "msort.h"

/* make a random array of n elements */
float * random_array(long n, long seed) {
  float * a = (float *)malloc(sizeof(float) * n);
  unsigned short rg[3] = { (unsigned short)((seed >> 32) & 65535),
                           (unsigned short)((seed >> 16) & 65535),
                           (unsigned short)((seed >> 0 ) & 65535) };
  for (long i = 0; i < n; i++) {
    a[i] = erand48(rg);
  }
  return a;
}

/* make a random array of n elements */
float * const_array(long n, float c) {
  float * a = (float *)malloc(sizeof(float) * n);
  for (long i = 0; i < n; i++) {
    a[i] = c;
  }
  return a;
}

/* check if a[0:n] is sorted */
int count_unsorted(float * a, long n) {
  int err = 0;
  for (long i = 0; i < n - 1; i++) {
    if (a[i] > a[i+1]) {
      fprintf(stderr, "a[%ld] = %f > a[%ld] = %f\n",
              i, a[i], i + 1, a[i + 1]);
      err++;
    }
    assert(a[i] <= a[i+1]);
  }
  return err;
}

int main(int argc, char ** argv) {
  int i = 1;
  long n              = (argc > i ? atol(argv[i]) : 100 * 1000 * 1000); i++;
  /* n <= ms_threshold -> insertion sort */
  long ms_threshold   = (argc > i ? atol(argv[i]) : 50);        i++;
  long mg_threshold   = (argc > i ? atol(argv[i]) : 10 * 1000); i++;
  long seed           = (argc > i ? atol(argv[i]) : 12345678);  i++;
  const char * exe = argv[0];
  const char * th_s = getenv("OMP_NUM_THREADS");
  long th = (th_s ? atol(th_s) : 1);
  const char * events = getenv("EV");
  if (!events) events = strdup("ref-cycles,cycles,instructions,L1-dcache-load-misses,cache-misses");
  float * a = random_array(n, seed);
  float * b = const_array(n, 0);
  perf_event_counters_t pc = mk_perf_event_counters(events);
  printf("exe: %s\n", exe);
  printf("threads: %ld\n", th);
  printf("elements: %ld\n", n);
  printf("threshold from merge to insertion: %ld\n", ms_threshold);
  printf("threshold from parallel merge to serial merge: %ld\n", mg_threshold);
  perf_event_values_t v0 = perf_event_counters_get(pc);
  /* real thing happens here */
  msort_from_main(a, b, a, 0, n, ms_threshold, mg_threshold);
  perf_event_values_t v1 = perf_event_counters_get(pc);
  for (int i = 0; i < pc.n; i++) {
    printf("%s : %lld\n", pc.events[i], v1.values[i] - v0.values[i]);
  }
  perf_event_counters_destroy(pc);
  long us = count_unsorted(a, n);
  if (us == 0) {
    printf("OK\n");
  } else {
    printf("NG\n");
  }
  return 0;
}

In [ ]:
%%writefile msort.h
#pragma once
/* merge a[p:q] and a[s:t] into b[d:..] */
void merge(float * a, float * b, long p, long q, long s, long t, long d, long th);

/* merge, called from main */
void merge_from_main(float * a, float * b, long p, long q, long s, long t, long d, long th);
  
/* merge sort, called from the main */
void msort_from_main(float * a, float * b, float * g, long p, long q, long th0, long th1);


* compile them together

In [ ]:
clang++ -O3 -Wall msort.cc merge.cc msort_main.cc -o msort_serial -lpfm
# nvc++ -O3 -Wall msort.cc merge.cc msort_main.cc -o msort_serial -lpfm

* run:
```
./msort_serial N
```
* will generate a random array of N elements and sort it
* default value for N is 100,000,000 (100M)

In [ ]:
./msort_serial $((100 * 1000 * 1000))

# <font color="green"> Problem 1 :  Parallelize merge sort</font>
* parallelize the merge sort algorithm by modifying the following cell
* it should be easy with task parallelism

In [ ]:
%%writefile msort_parallel.cc
#com 2
#include "msort.h"

/* insertion sort for leaf cases */
/* return m a[m] = min {a[begin:end]} */ 
static long choose_min_idx(float * a, long begin, long end) {
  long m = begin;
  for (long i = begin + 1; i < end; i++) {
    if (a[i] < a[m]) {
      m = i;
    }
  }
  return m;
}

/* sort a[p:q] by insertion sort */
static void insertion_sort(float * a, long p, long q) {
  for (long i = p; i < q; i++) {
    long j = choose_min_idx(a, i, q);
    float t = a[i];
    a[i] = a[j];
    a[j] = t;
  }
}

/* sort a[p:q] into g[p:q], using b[p:q] as a temporary space */
static void msort_r(float * a, float * b, float * g, long p, long q, long th0, long th1) {
  if (q - p < th0) {
    /* the array is small -> switch to insertion sort */
    if (g != a) {
      for (long i = p; i < q; i++) {
        g[i] = a[i];
      }
    }
    insertion_sort(g, p, q);
  } else {
    long r = (p + q) / 2;
    /* get partial results into the other array != g */
    float * h = (g == a ? b : a);
#ifpy VER == 2
#pragma omp task
#endifpy
    msort_r(a, b, h, p, r, th0, th1);
#ifpy VER == 2
#pragma omp task
#endifpy
    msort_r(a, b, h, r, q, th0, th1);
#ifpy VER == 2
#pragma omp taskwait
#endifpy
    /* merge h[p:r] and h[r:q] -> g[p:]*/
    merge(h, g, p, r, r, q, p, th1);
  }
}

/* merge sort, called from the main */
void msort_from_main(float * a, float * b, float * g, long p, long q, long th0, long th1) {
#ifpy VER == 2
#pragma omp parallel
#pragma omp master
#endifpy
  msort_r(a, b, g, p, q, th0, th1);
}


In [ ]:
clang++ -O3 -Wall -fopenmp msort_parallel.cc merge.cc msort_main.cc -o msort_parallel -lpfm
# nvc++ -O3 -Wall      -mp msort_parallel.cc merge.cc msort_main.cc -o msort_parallel -lpfm

* run the following cell with various values for `OMP_NUM_THREADS` and the number of elements and see how it scales

In [ ]:
OMP_NUM_THREADS=1 ./msort_parallel $((100 * 1000 * 1000))

* you can check how many processors does the host have by 

In [ ]:
lscpu | grep CPU

* here is a script to run it with various number of threads

In [ ]:
#!/usr/bin/python3
import os

def run1(cmd):
    print(cmd)
    return os.system(cmd)

def run(exes, sizes, threads, repeats=3):
    os.makedirs("out", exist_ok=True)
    for exe in exes:
        for n in sizes:
            for h in threads:
                for t in range(repeats):
                    r = run1("OMP_NUM_THREADS={h} ./{exe} {n} > out/out_{exe}_{h}_{t}.txt"
                             .format(exe=exe, n=n, h=h, t=t))
                    if r:
                        return r
    print("done")
    return 0


if 0:
    run(["msort_2_1.exe", "msort_2_2.exe"],
        [ 10 * 1000 * 1000 ],
        [ 1, 2, 3, 4, 6, 8 ])
    

In [ ]:
run(["msort_parallel"],
    [ 10 * 1000 * 1000 ],
    [ 1, 2, 3, 4, 6, 8, 12, 16, 20, 24, 32 ])

* visualize it

In [ ]:
#!/usr/bin/python3
import re
import pandas as pd

def mk_key(s):
    s = s.strip()
    s = s.replace(" ", "_")
    s = s.replace("-", "_")
    return s

def mk_val(s):
    try:
        return int(s)
    except ValueError:
        pass
    try:
        return float(s)
    except ValueError:
        pass
    return s.strip()

def read_file(filename):
    data = {}
    pat = re.compile("(?P<key>[^:]+):(?P<val>[^:]+)")
    with open(filename) as fp:
        for line in fp:
            m = pat.match(line)
            if m:
                key = mk_key(m.group("key"))
                val = mk_val(m.group("val"))
                data[key] = val
    return data

def read_files(filenames):
    data = []
    for filename in filenames:
        data.append(read_file(filename))
    return pd.DataFrame(data)



In [ ]:
#!/usr/bin/python3
import glob
import matplotlib.pyplot as plt
import numpy as np

#from read_data import read_files

def speedup(filenames, exes=None):
    # read all data
    dfa = read_files(filenames)
    fig, ax = plt.subplots()
    if exes is None:
        exes = dfa["exe"].unique()
    exe0 = exes[0]
    base = dfa[(dfa["exe"] == exe0) & (dfa["threads"] == 1)]["ref_cycles"].agg("mean")
    for exe in exes:
        df = dfa[dfa["exe"] == exe]
        avg = df.groupby(["threads"]).mean()
        avg = avg.sort_values("threads")
        threads = np.array(avg.index)
        perf = base / np.array(avg["ref_cycles"])
        label = exe.strip()
        line, = ax.plot(threads, perf, "*-", label=label)
    ax.set_xlabel("threads")
    ax.set_ylabel("speedup")
    plt.legend()
    plt.xlim(0)
    plt.ylim(0)
    plt.show()

if 0:
    speedup(glob.glob("versioned/out/out_*.txt"))
    

In [ ]:
speedup(glob.glob("out/out_*.txt"), exes=["./msort_parallel"])

# 3. Parallelize merge 
* serial merge sort has $\Theta(n\log n)$ time-complexity
* executing two recursive calls in parallel does not attain enough speedup as the algorithm still has a critical path of $\Theta(n)$
* the reason for this is the serial merge step
* to make the critical path sublinear $o(n)$, it is important to parallelize the merge step itself
* let's first make a program that only merges two sorted arrays

In [ ]:
%%writefile merge_main.cc
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include "event.h"
#include "msort.h"

float * random_sorted_two_arrays(long n, long seed) {
  float * a = (float *)malloc(sizeof(float) * n);
  unsigned short rg[3] = { (unsigned short)((seed >> 32) & 65535),
                           (unsigned short)((seed >> 16) & 65535),
                           (unsigned short)((seed >> 0 ) & 65535) };
  long h = n / 2;
  float s = 0.0;
  for (long i = 0; i < h; i++) {
    s += erand48(rg) / h;
    a[i] = s;
  }
  s = 0.0;
  for (long i = h; i < n; i++) {
    s += erand48(rg) / h;
    a[i] = s;
  }
  return a;
}

/* make a random array of n elements */
float * const_array(long n, float c) {
  float * a = (float *)malloc(sizeof(float) * n);
  for (long i = 0; i < n; i++) {
    a[i] = c;
  }
  return a;
}

/* check if a[0:n] is sorted */
int count_unsorted(float * a, long n) {
  int err = 0;
  for (long i = 0; i < n - 1; i++) {
    if (a[i] > a[i+1]) {
      fprintf(stderr, "a[%ld] = %f > a[%ld] = %f\n",
              i, a[i], i + 1, a[i + 1]);
      err++;
    }
    assert(a[i] <= a[i+1]);
  }
  return err;
}

int main(int argc, char ** argv) {
  int i = 1;
  long n              = (argc > i ? atol(argv[i]) : 100 * 1000 * 1000); i++;
  long mg_threshold   = (argc > i ? atol(argv[i]) :  10 * 1000);        i++;
  long seed           = (argc > i ? atol(argv[i]) : 12345678); i++;
  const char * events = getenv("EV");
  if (!events) events = strdup("ref-cycles,cycles,instructions,L1-dcache-load-misses,cache-misses");
  long h = n / 2;
  float * a = random_sorted_two_arrays(n, seed);
  float * b = const_array(n, 0);
  perf_event_counters_t pc = mk_perf_event_counters(events);
  printf("merge %ld + %ld elements\n", h, n - h);
  perf_event_values_t v0 = perf_event_counters_get(pc);
  /* real thing happens here */
  merge_from_main(a, b, 0, h, h, n, 0, mg_threshold);
  perf_event_values_t v1 = perf_event_counters_get(pc);
  for (int i = 0; i < pc.n; i++) {
    printf("%s : %lld\n", pc.events[i], v1.values[i] - v0.values[i]);
  }
  perf_event_counters_destroy(pc);
  long us = count_unsorted(b, n);
  if (us == 0) {
    printf("OK\n");
  } else {
    printf("NG\n");
  }
  return 0;
}

In [ ]:
clang++ -O3 -Wall -fopenmp merge.cc merge_main.cc -o merge_serial -lpfm
# nvc++ -O3 -Wall      -mp merge.cc merge_main.cc -o merge_serial -lpfm

In [ ]:
./merge_serial $((100 * 1000 * 1000))

# <font color="green"> Problem 2 :  Parallelize merge</font>
* parallelize the merge algorithm by modifying the following cell
* hint: here again, divide-and-conquer is useful
* let's say we merge two arrays $a$ and $b$ into another array $c$
1. split $a$ or $b$ (the larger one) into two halves
1. assume wolg $a$ is the larger and call the first and second half $a_0$ and $a_1$, respectively
1. let $p$ be the smallest value of $a_1$
1. split $b$ into two parts, $b_0$, which are elements $\leq p$ and $b_1$, elements $\geq p$ (use binary search to find the boundary between them)
1. merge $a_0$ and $b_0$; merge $a_1$ and $b_1$ (two merges can be done in parallel)

In [ ]:
%%writefile merge_parallel.cc
#com 2
#include <assert.h>
#include "msort.h"

#ifpy VER == 1
void merge(float * a, float * b, long p, long q, long s, long t, long d, long th) {
  (void)th;
  long i = p;
  long j = s;
  long k = d;
  while (i < q && j < t) {
    if (a[i] < a[j]) {
      b[k++] = a[i++];
    } else {
      b[k++] = a[j++];
    }
  }
  while (i < q) {
    b[k++] = a[i++];
  }
  while (j < t) {
    b[k++] = a[j++];
  }
}

/* merge, called from main */
void merge_from_main(float * a, float * b, long p, long q, long s, long t, long d, long th) {
  merge(a, b, p, q, s, t, d, th);
}

#elsepy

/* merge a[p:q] and a[r:s] --> b[d:] */
static void merge_s(float * a, float * b, long p, long q, long s, long t, long d) {
  long i = p;
  long j = s;
  long k = d;
  while (i < q && j < t) {
    if (a[i] < a[j]) {
      b[k++] = a[i++];
    } else {
      b[k++] = a[j++];
    }
  }
  while (i < q) {
    b[k++] = a[i++];
  }
  while (j < t) {
    b[k++] = a[j++];
  }
}

/* find r s.t. a[r-1] <= piv < a[r] */
static long find_piv(float * a, long b, long e, float piv) {
  if (piv < a[b]) return b;
  if (a[e - 1] <= piv) return e;
  long p = b;
  long q = e - 1;
  /* a[p] <= piv < a[q] */
  while (q - p > 1) {
    assert(a[p] <= piv);
    assert(piv < a[q]);
    long r = (p + q) / 2;
    if (a[r] <= piv) {
      p = r;                    // a[p] <= piv
    } else {
      q = r;                    // piv < a[q]
    }
  }
  assert(a[q - 1] <= piv);
  assert(piv < a[q]);
  /* a[q - 1] <= piv < a[q] */
  return q;
}

/* merge a[p:q] and a[r:s] --> b[d:] */
void merge(float * a, float * b, long p, long q, long s, long t, long d, long th) {
  if (q - p + t - s < th) {
    merge_s(a, b, p, q, s, t, d);
  } else if (q - p > t - s) {
    long r = (p + q) / 2;
    float piv = a[r];
    long u = find_piv(a, s, t, piv);
#pragma omp task
    merge(a, b, p, r, s, u, d, th);
#pragma omp task
    merge(a, b, r, q, u, t, d + (r - p) + (u - s), th);
#pragma omp taskwait
  } else {
    long u = (s + t) / 2;
    float piv = a[u];
    long r = find_piv(a, p, q, piv);
#pragma omp task
    merge(a, b, p, r, s, u, d, th);
#pragma omp task
    merge(a, b, r, q, u, t, d + (r - p) + (u - s), th);
#pragma omp taskwait
  }
}

/* merge, called from main */
void merge_from_main(float * a, float * b, long p, long q, long s, long t, long d, long th) {
#pragma omp parallel
#pragma omp master
  merge(a, b, p, q, s, t, d, th);
}

#endifpy

In [ ]:
clang++ -O3 -Wall -fopenmp merge_parallel.cc merge_main.cc -o merge_parallel -lpfm
# nvc++ -O3 -Wall      -mp merge_parallel.cc merge_main.cc -o merge_parallel -lpfm

* run the following cell with various values for `OMP_NUM_THREADS` and the number of elements and see how it scales

In [ ]:
OMP_NUM_THREADS=1 ./merge_parallel $((100 * 1000 * 1000))

# <font color="green"> Problem 3 :  Parallel merge sort with parallel merge</font>
* use the parallel merge in the parallel merge sort and see the scalability

In [ ]:
clang++ -O3 -Wall -fopenmp msort_parallel.cc merge_parallel.cc msort_main.cc -o msort_parallel2 -lpfm
# nvc++ -O3 -Wall      -mp msort_parallel.cc merge_parallel.cc msort_main.cc -o msort_parallel2 -lpfm

* run the following cell with various values for `OMP_NUM_THREADS` and the number of elements and see how it scales

In [ ]:
OMP_NUM_THREADS=1 ./msort_parallel2 $((100 * 1000 * 1000))

* run it with various number of threads

In [ ]:
run(["msort_parallel2"],
    [ 10 * 1000 * 1000 ],
    [ 1, 2, 3, 4, 6, 8, 12, 16, 20, 24, 32 ])

In [ ]:
speedup(glob.glob("out/out_*.txt"), exes=["./msort_parallel", "./msort_parallel2"])